# 準備

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

#モデル
import lightgbm as lgb

#パラメータ探索
import optuna

#クロスバリデーション
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

# 評価指標
import sys
sys.path.append('../evaluation')
from rmse import rmse
from recall5 import recall5
from profit import profit

# #エヴァリュエーション
# from sklearn.metrics import precision_score, recall_score, mean_squared_error

#可視化
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report

#保存
import pickle

In [2]:
#データを読み込む
n_uma_race_df = pd.read_pickle('../../datasets/traindata/n_uma_race.pkl')
n_race_df = pd.read_pickle('../../datasets/traindata/n_race.pkl')
n_payout_df = pd.read_pickle('../../datasets/traindata/n_payout.pkl')

In [3]:
# 新しいグループを作成
n_uma_race_df['group'] = n_uma_race_df['kaisai_nen'].astype(int).astype(str) +"-"+ n_uma_race_df['kaisai_tsukihi'].astype(int).astype(str) +"-"+  n_uma_race_df['keibajo_code'].astype(int).astype(str) +"-"+  n_uma_race_df['race_bango'].astype(int).astype(str)
n_race_df['group'] = n_race_df['kaisai_nen'].astype(int).astype(str) +"-"+ n_race_df['kaisai_tsukihi'].astype(int).astype(str) +"-"+  n_race_df['keibajo_code'].astype(int).astype(str) +"-"+  n_race_df['race_bango'].astype(int).astype(str)
n_payout_df['group'] = n_payout_df['kaisai_nen'].astype(int).astype(str) +"-"+ n_payout_df['kaisai_tsukihi'].astype(int).astype(str) +"-"+  n_payout_df['keibajo_code'].astype(int).astype(str) +"-"+  n_payout_df['race_bango'].astype(int).astype(str)

n_race_df_drop = n_race_df.drop(['kaisai_nen', 'kaisai_tsukihi', 'keibajo_code', 'kaisai_kai', 'kaisai_nichime', 'race_bango'],axis=1)
n_payout_df_drop = n_payout_df.drop(['kaisai_nen', 'kaisai_tsukihi', 'keibajo_code', 'kaisai_kai', 'kaisai_nichime', 'race_bango', 'toroku_tosu', 'shusso_tosu'],axis=1)

merged_df = pd.merge(n_uma_race_df, n_race_df_drop, on='group', how='left')
merged_df = pd.merge(merged_df, n_payout_df_drop, on='group', how='left')
merged_df['group'].nunique()

205318

In [4]:
merged_df['group'].unique()

array(['2023-916-46-1', '2023-916-46-2', '2023-916-46-3', ...,
       '2023-907-50-10', '2023-907-50-11', '2023-907-50-12'], dtype=object)

In [5]:
pd.set_option('display.max_columns', None)
merged_df.head(1)

,kaisai_nen,kaisai_tsukihi,keibajo_code,kaisai_kai,kaisai_nichime,race_bango,wakuban,umaban,ketto_toroku_bango,bamei,umakigo_code,seibetsu_code,hinshu_code,moshoku_code,barei,tozai_shozoku_code,chokyoshi_code,banushi_code,banushimei,futan_juryo,blinker_shiyo_kubun,kishu_code,kishumei_ryakusho,kishu_minarai_code,bataiju,zogen_fugo,zogen_sa,ijo_kubun_code,nyusen_juni,kakutei_chakujun,dochaku_kubun,dochaku_tosu,soha_time,chakusa_code_1,chakusa_code_2,chakusa_code_3,corner_1,corner_2,corner_3,corner_4,tansho_odds,tansho_ninkijun,kakutoku_honshokin,kakutoku_fukashokin,kohan_4f,kohan_3f,aiteuma_joho_1,aiteuma_joho_2,aiteuma_joho_3,time_sa,record_koshin_kubun,kyakushitsu_hantei,group,yobi_code,jusho_kaiji,grade_code,kyoso_shubetsu_code,kyoso_kigo_code,juryo_shubetsu_code,kyoso_joken_code_2sai,kyoso_joken_code_3sai,kyoso_joken_code_4sai,kyoso_joken_code_5sai_ijo,kyoso_joken_code,kyori,track_code,course_kubun,honshokin,fukashokin,hasso_jikoku,toroku_tosu,shusso_tosu,nyusen_tosu,tenko_code,babajotai_code_shiba,babajotai_code_dirt,fuseiritsu_flag_sanrenpuku,tokubarai_flag_sanrenpuku,henkan_flag_sanrenpuku,haraimodoshi_sanrenpuku_1a,haraimodoshi_sanrenpuku_1b,haraimodoshi_sanrenpuku_1c
0,2023,916,46,13,1,1,1,1,2020106227,スターサファイア,0,1,1.0,3,3,3,5319,875800,ＪＰＮ技研,560.0,0,5300,葛山晃平,0,466.0,1,0.0,0,8,8,0,0,1359,4.0,NaN,NaN,9,9,9,9,899,8,0,0,0,411,2020105068フェイマスグローリ,0000000000,0000000000,50,0,0,2023-916-46-1,1,0,0,49,0,4,0,0,0,0,0,1400,24,0,4.000000e+51,0,1140,9,9,9,1,0,1.0,0.0,0.0,0.0,60809.0,210.0,1.0


# 前処理

In [6]:
merged_df['hutan_wariai'] = merged_df['futan_juryo'] / merged_df['bataiju']

In [7]:
def combine_sign_and_diff(row):
    if row['zogen_fugo'] == 2:
        return row['zogen_sa']
    elif row['zogen_fugo'] == 0:
        return -row['zogen_sa']
    else:
        return 0

merged_df['zogen_ryou'] = merged_df.apply(combine_sign_and_diff, axis=1)

In [8]:
merged_df = merged_df[merged_df['ijo_kubun_code'] == 0]
# 1	出走取消	取消	SCRATCHED	S

In [9]:
columns_to_convert = [
                    'wakuban',
                    'umaban',

                    'kyori',
                    'grade_code',
                    'seibetsu_code',
                    'moshoku_code',
                    'barei',
                    'chokyoshi_code',
                    'banushi_code',
                    'kishu_code',
                    'kishu_minarai_code',
                    'kyoso_shubetsu_code',
                    'juryo_shubetsu_code',
                    'shusso_tosu',
                    'tenko_code',
                    'babajotai_code_dirt',
                    'hutan_wariai',
                    'zogen_ryou',
                    'track_code',
                    'keibajo_code',
                    'hinshu_code',
                    'umakigo_code'
                    ]

for column in columns_to_convert:
    merged_df[column].fillna(0, inplace=True)
    try:
        if merged_df[column].astype(float).apply(lambda x: x.is_integer()).all():
            merged_df[column] = merged_df[column].astype(int)
        else:
            merged_df[column] = merged_df[column].astype(float)
    except ValueError:
        merged_df[column] = merged_df[column].astype(float)

    print(f"{column} のデータ型: {merged_df[column].dtype}")

wakuban のデータ型: int32
umaban のデータ型: int32
kyori のデータ型: int32
grade_code のデータ型: int32
seibetsu_code のデータ型: int32
moshoku_code のデータ型: int32
barei のデータ型: int32
chokyoshi_code のデータ型: int32
banushi_code のデータ型: int32
kishu_code のデータ型: int32
kishu_minarai_code のデータ型: int32
kyoso_shubetsu_code のデータ型: int32
juryo_shubetsu_code のデータ型: int32
shusso_tosu のデータ型: int32
tenko_code のデータ型: int32
babajotai_code_dirt のデータ型: int32
hutan_wariai のデータ型: float64
zogen_ryou のデータ型: int32
track_code のデータ型: int32
keibajo_code のデータ型: int32
hinshu_code のデータ型: int32
umakigo_code のデータ型: int32


In [10]:
merged_df[column].info()

<class 'pandas.core.series.Series'>
Int64Index: 2017402 entries, 0 to 2042540
Series name: umakigo_code
Non-Null Count    Dtype
--------------    -----
2017402 non-null  int32
dtypes: int32(1)
memory usage: 23.1 MB


In [11]:
def invert_rankings(df, column_name='kakutei_chakujun'):
    """
    この関数はDataFrameの特定の列のランキングを逆転させます。
    - df: DataFrameオブジェクト
    - column_name: 逆転させる列の名前（デフォルトは 'kakutei_chakujun'）
    """
    max_rank = df.groupby('group')[column_name].transform('max')
    df[column_name] = max_rank - df[column_name] + 1
    
    return df

In [12]:
def modify_rankings(df, column_name='kakutei_chakujun'):
    """
    この関数はDataFrameの特定の列のランキングを修正します。
    - df: DataFrameオブジェクト
    - column_name: 修正する列の名前（デフォルトは 'kakutei_chakujun'）
    """
    conditions = [
        (df[column_name] == 1),
        (df[column_name] == 2),
        (df[column_name] == 3),
        (df[column_name] == 4),
        (df[column_name] == 5)
    ]
    
    values = [5, 4, 3, 2, 1]
    
    df['target'] = np.select(conditions, values, default=0)
    
    return df

In [13]:
merged_df = modify_rankings(merged_df)

In [14]:
merged_df['target'].value_counts()

0    999416
5    203842
4    203673
3    203665
2    203430
1    203376
Name: target, dtype: int64

# lightgbm.LGBMRanker

In [15]:
# 2010年から2019年のデータを学習データとして取得
train_data = merged_df[merged_df['kaisai_nen'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])]
test_data = merged_df[merged_df['kaisai_nen'].isin([2020, 2021, 2022])]

In [16]:
# ランキング学習のために必要な特徴量とターゲットを設定
features = [
            # 'wakuban',
            'umaban',

            # 'seibetsu_code',
            # 'moshoku_code',
            # 'barei',
            # 'chokyoshi_code',
            # 'banushi_code',
            # 'kishu_code',
            # 'kishu_minarai_code',
            # 'hutan_wariai',
            'zogen_ryou',
            # 'hinshu_code',
            # 'umakigo_code',
            ]

target = 'target'

In [17]:
categorical_features = [
                        # 'seibetsu_code',
                        # 'moshoku_code',
                        # 'chokyoshi_code',
                        # 'banushi_code',
                        # 'kishu_code',
                        # 'kishu_minarai_code',
                        # 'hinshu_code',
                        # 'umakigo_code',
                        ]

In [18]:
def ndcg_score(y_true, y_score, k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

def mean_ndcg_score(y_true, y_score, groups, k=5):
    ndcgs = []
    idx_start = 0
    for group in groups:
        ndcgs.append(ndcg_score(y_true[idx_start:idx_start+group], y_score[idx_start:idx_start+group], k))
        idx_start += group
    return np.mean(ndcgs)

In [19]:
group_sizes = train_data.groupby('group').size()

def objective(trial):
    # Optuna parameters
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_at': 5,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'verbose': trial.suggest_int('verbose', 0, 1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-5, 10.0, log=True),  # L1正則化
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-5, 10.0, log=True)  # L2正則化
    }
    
    gkf = GroupKFold(n_splits=5)
    ndcgs = []
    
    for train_index, valid_index in gkf.split(train_data, groups=train_data['group']):
        train_fold_data = train_data.iloc[train_index]
        valid_fold_data = train_data.iloc[valid_index]
        for feature in categorical_features:
            train_fold_data = train_fold_data.copy()
            valid_fold_data = valid_fold_data.copy()
            train_fold_data[feature] = train_fold_data[feature].astype('category')
            valid_fold_data[feature] = valid_fold_data[feature].astype('category')

        train_fold_group_sizes = train_fold_data.groupby('group').size().tolist()
        valid_fold_group_sizes = valid_fold_data.groupby('group').size().tolist()

        ranker = lgb.LGBMRanker(**params)
        ranker.fit(train_fold_data[features], train_fold_data[target], 
           group=train_fold_group_sizes, 
           eval_set=[(valid_fold_data[features], valid_fold_data[target])], 
           eval_group=[valid_fold_group_sizes], categorical_feature=categorical_features) 
        
        y_pred = ranker.predict(valid_fold_data[features])
        ndcg_value = mean_ndcg_score(valid_fold_data[target].values, y_pred, valid_fold_group_sizes)
        ndcgs.append(ndcg_value)
    
    return np.mean(ndcgs)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

[I 2023-11-06 16:25:02,504] A new study created in memory with name: no-name-20116ce5-3245-44c1-a725-b89996fc0966
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] feature_fraction is set=0.7113479132324596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7113479132324596
[LightGBM] [Warning] lambda_l1 is set=3.7795848495489845e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7795848495489845e-05
[LightGBM] [Warning] lambda_l2 is set=0.1778180298410174, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1778180298410174
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.7113479132324596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7113479132324596
[LightGBM] [Warning] lambda_l1 is set=3.7795848495489845e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7795848495489845e-05
[LightGBM] [Warning] lambda_l2 is set=0.1778180298410174, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1778180298410174
[LightGBM] [Warning] bagging_freq 

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] feature_fraction is set=0.7113479132324596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7113479132324596
[LightGBM] [Warning] lambda_l1 is set=3.7795848495489845e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7795848495489845e-05
[LightGBM] [Warning] lambda_l2 is set=0.1778180298410174, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1778180298410174
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.7113479132324596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7113479132324596
[LightGBM] [Warning] lambda_l1 is set=3.7795848495489845e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.7795848495489845e-05
[LightGBM] [Warning] lambda_l2 is set=0.1778180298410174, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1778180298410174
[LightGBM] [Warning] bagging_freq 

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


...

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] feature_fraction is set=0.9567831154498239, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9567831154498239
[LightGBM] [Warning] lambda_l1 is set=0.7919856183140275, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7919856183140275
[LightGBM] [Warning] lambda_l2 is set=0.004234111169157586, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004234111169157586
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9567831154498239, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9567831154498239
[LightGBM] [Warning] lambda_l1 is set=0.7919856183140275, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7919856183140275
[LightGBM] [Warning] lambda_l2 is set=0.004234111169157586, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004234111169157586
[LightGBM] [Warning] bagging_freq is set=1

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] feature_fraction is set=0.9567831154498239, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9567831154498239
[LightGBM] [Warning] lambda_l1 is set=0.7919856183140275, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7919856183140275
[LightGBM] [Warning] lambda_l2 is set=0.004234111169157586, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004234111169157586
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9567831154498239, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9567831154498239
[LightGBM] [Warning] lambda_l1 is set=0.7919856183140275, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7919856183140275
[LightGBM] [Warning] lambda_l2 is set=0.004234111169157586, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004234111169157586
[LightGBM] [Warning] bagging_freq is set=1

[I 2023-11-06 16:30:30,182] Trial 2 finished with value: 17.26865312879388 and parameters: {'num_leaves': 237, 'learning_rate': 0.0002491364683993565, 'feature_fraction': 0.9567831154498239, 'bagging_freq': 1, 'verbose': 1, 'lambda_l1': 0.7919856183140275, 'lambda_l2': 0.004234111169157586}. Best is trial 1 with value: 17.290179868586876.


In [20]:
# 最適なパラメータ
best_params = study.best_params

# KFoldでのモデル訓練
kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []

for train_index, valid_index in kf.split(train_data):
    train_fold_data = train_data.iloc[train_index]
    valid_fold_data = train_data.iloc[valid_index]
    
    train_fold_group_sizes = train_fold_data.groupby('group').size().tolist()
    valid_fold_group_sizes = valid_fold_data.groupby('group').size().tolist()
    
    ranker = lgb.LGBMRanker(**best_params)
    ranker.fit(
        train_fold_data[features], train_fold_data[target], 
        group=train_fold_group_sizes, 
        eval_set=[(valid_fold_data[features], valid_fold_data[target])], 
        eval_group=[valid_fold_group_sizes], 
        eval_at=5, early_stopping_rounds=20, verbose=10
    )
    models.append(ranker)

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsam

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsam

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsam

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsam

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7865838919063934, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7865838919063934
[LightGBM] [Warning] lambda_l1 is set=1.8988063368464292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8988063368464292
[LightGBM] [Warning] lambda_l2 is set=0.8775552989184024, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8775552989184024
[LightGBM] [Warning] bagging_freq is set=2, subsam

# モデル保存

In [21]:
# 学習
full_train_dataset = lgb.Dataset(merged_df[features], label=merged_df[target])
full_model = lgb.train(best_params, full_train_dataset, num_boost_round=500, verbose_eval=100)

# モデルを保存
model_save_path = '../../models/model.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(full_model, f)

print(f"Model saved to {model_save_path}")

c:\Users\kawam\miniforge3\envs\keiba\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107
[LightGBM] [Info] Number of data points in the train set: 2017402, number of used features: 2
[LightGBM] [Info] Start training from score 1.514390
Model saved to ../../models/model.pkl
